In [ ]:
from fuzzingbook.MutationFuzzer import MutationFuzzer

seed_input = "http://www.google.com/search?q=fuzzing"
mutation_fuzzer = MutationFuzzer(seed=[seed_input])
seeds = [mutation_fuzzer.fuzz() for i in range(10)]
print(seeds)


In [ ]:
from fuzzingbook.MutationAnalysis import MuFunctionAnalyzer, MuProgramAnalyzer
from fuzzingbook.ExpectError import ExpectError
import sys

In [ ]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'Equilateral'
        else:
            return 'Isosceles'
    else:
        if b == c:
            return "Isosceles"
        else:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"

In [ ]:
def weak_oracle(fn):
    assert fn(1, 1, 1) == 'Equilateral'

    assert fn(1, 2, 1) != 'Equilateral'
    assert fn(2, 2, 1) != 'Equilateral'
    assert fn(1, 2, 2) != 'Equilateral'

    assert fn(1, 2, 3) != 'Equilateral'

In [ ]:
def strong_oracle(fn):
    assert fn(1, 1, 1) == 'Equilateral'

    assert fn(1, 2, 1) == 'Isosceles'
    assert fn(2, 2, 1) == 'Isosceles'
    assert fn(1, 2, 2) == 'Isosceles'

    assert fn(1, 2, 3) == 'Scalene'


In [ ]:
strong_oracle(triangle)

In [ ]:
for mutant in MuFunctionAnalyzer(triangle):
    with ExpectError():
        with mutant:
            weak_oracle(triangle)

In [ ]:
MuFunctionAnalyzer(triangle).nmutations

In [ ]:
x = MuFunctionAnalyzer(triangle)

In [ ]:
for mutant in x:
    with mutant:
        strong_oracle(triangle)
print(mutant .pm.score())

In [ ]:

(x.get_mutation_count())

In [ ]:
for mutant in MuFunctionAnalyzer(triangle, log=True):
    with mutant:
        assert triangle(1, 1, 1) == 'Equilateral', "Equal Check1"
        assert triangle(1, 0, 1) != 'Equilateral', "Equal Check2"
        assert triangle(1, 0, 2) != 'Equilateral', "Equal Check3"
mutant.pm.score()

In [ ]:
def gcd(a, b):
    if a < b:
        c = a
        a = b
        b = c

    while b != 0:
        c = a
        a = b
        b = c % b

    return a

In [ ]:
for mutant in MuFunctionAnalyzer(gcd, log=True):
    with mutant:
        assert gcd(1, 0) == 1, "Minimal"
        assert gcd(0, 1) == 1, "Mirror"
print(mutant.pm.score())

# Modules

In [ ]:
import types
import inspect
def import_code(code, name):
    module = types.ModuleType(name)
    exec(code, module.__dict__)
    return module
shape_src = inspect.getsource(triangle)
shape = import_code(shape_src, 'shape')

In [ ]:
shape.triangle(1,2,3)

In [ ]:
import unittest
class StrongShapeTest(unittest.TestCase):

    def test_equilateral(self):
        assert shape.triangle(1, 1, 1) == 'Equilateral'

    def test_isosceles(self):
        assert shape.triangle(1, 2, 1) == 'Isosceles'
        assert shape.triangle(2, 2, 1) == 'Isosceles'
        assert shape.triangle(1, 2, 2) == 'Isosceles'

    def test_scalene(self):
        assert shape.triangle(1, 2, 3) == 'Scalene'

        
class WeakShapeTest(unittest.TestCase):
    def test_equilateral(self):
        assert shape.triangle(1, 1, 1) == 'Equilateral'

    def test_isosceles(self):
        assert shape.triangle(1, 2, 1) != 'Equilateral'
        assert shape.triangle(2, 2, 1) != 'Equilateral'
        assert shape.triangle(1, 2, 2) != 'Equilateral'

    def test_scalene(self):
        assert shape.triangle(1, 2, 3) != 'Equilateral'
        
def suite(test_class):
    suite = unittest.TestSuite()
    for f in test_class.__dict__:
        if f.startswith('test_'):
            suite.addTest(test_class(f))
    return suite


In [ ]:
suite(StrongShapeTest).run(unittest.TestResult())

In [ ]:
MuProgramAnalyzer('shape', shape_src).nmutations

In [ ]:
for m in MuProgramAnalyzer('shape', shape_src):
    print(m.name)

In [ ]:
for mutant in MuProgramAnalyzer('shape', shape_src):
    print(mutant.name)
    print(mutant.diff())
    break

In [ ]:
test_module = sys.modules[__name__]
for mutant in MuProgramAnalyzer('shape', shape_src):
    mutant[test_module].runTest('WeakShapeTest')
mutant.pm.score()

## gcd function

In [4]:
def gcd(a, b):
    if a < b:
        c = a
        a = b
        b = c

    while b != 0:
        c = a
        a = b
        b = c % b

    return a

In [ ]:
src = inspect.getsource(gcd)
program = import_code(src, 'gcd')

In [ ]:
MuProgramAnalyzer('program', src).nmutations

In [1]:
import unittest

class TestGCD(unittest.TestCase):
    def test_simple(self):
        assert function.gcd(1, 0) == 1

    def test_mirror(self):
        assert function.gcd(0, 1) == 1

# Function to get mutation score


In [7]:
# First define the function and test_name

import types
import inspect
from fuzzingbook.MutationAnalysis import MuProgramAnalyzer
from fuzzingbook.ExpectError import ExpectError
import sys

test_module = sys.modules[__name__]

def get_mutation_score(function, test_name):
    src = inspect.getsource(function)
    for mutant in MuProgramAnalyzer('function', src):
        mutant[test_module].runTest(test_name)
    return mutant.pm.score()

In [6]:
get_mutation_score(gcd, 'TestGCD')

ERROR: test_mirror (__main__.TestGCD)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/py/xtpzsqdd0tz15w13b3wtkqkm0000gn/T/ipykernel_38843/1753461744.py", line 8, in test_mirror
    assert function.gcd(0, 1) == 1
  File "<string>", line 5, in gcd
UnboundLocalError: local variable 'c' referenced before assignment

----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (errors=1)
FAIL: test_mirror (__main__.TestGCD)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/py/xtpzsqdd0tz15w13b3wtkqkm0000gn/T/ipykernel_38843/1753461744.py", line 8, in test_mirror
    assert function.gcd(0, 1) == 1
AssertionError

----------------------------------------------------------------------
Ran 2 tests in 0.000s

FAILED (failures=1)
----------------------------------------------------------------------
Ran 

0.42857142857142855